In [ ]:
"""
Created on Mon Jul 24 11:36 2023

This is a script to prepare the 50 km in front of ice shelf based on BedMachine

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
#from tqdm.notebook import tqdm
from tqdm import tqdm
import gsw
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.melt_functions as meltf
import basal_melt_param.box_functions as bf

from scipy.spatial import cKDTree

import itertools

import glob

In [ ]:
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
outputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_isf+'BedMachine_8km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)


In [ ]:
lon = file_isf['longitude']
lat = file_isf['latitude']

In [ ]:
file_BedMachine_orig = xr.open_dataset(inputpath_BedMachine+'BedMachine_v3_aggregated8km_allvars.nc')
file_mask = file_BedMachine_orig.sel(x=file_isf.x,y=file_isf.y)

In [ ]:
ocean = file_mask['ocean_conc'] > 0
# only points below 1500 m
offshore = file_mask['bed'] < -1500 # .drop('lon').drop('lat')
# only points above 1500 m
contshelf = file_mask['bed'] >= -1500 # .drop('lon').drop('lat')

In [ ]:
mask_50km = (ocean & contshelf).load()

lon_box = np.array([10.0])
lat_box = np.array([3.5])

close_region_around_isf_mask = tspf.mask_boxes_around_IF_new(lon, lat, mask_50km, 
                                file_isf['front_min_lon'], file_isf['front_max_lon'], 
                                file_isf['front_min_lat'], file_isf['front_max_lat'],  
                                lon_box, lat_box, 
                                file_isf['isf_name'])

In [ ]:
def distance_isf_points_from_line_small_domain(isf_points_da,line_points_da):
    
    """
    Compute the distance between ice shelf points and a line.
    
    This function computes the distance between ice shelf points and a line. This line can be the grounding
    line or the ice shelf front.
    
    Parameters
    ----------
    whole_domain : xarray.DataArray
        ice-shelf mask - all ice shelves are represented by a number, all other points (ocean, land) set to nan
    isf_points_da : xarray.DataArray
        array containing only points from one ice shelf
    line_points_da : xarray.DataArray
        mask representing the grounding line or ice shelf front mask corresponding to the ice shelf selected in ``isf_points_da``
        
    Returns
    -------
    xr_dist_to_line : xarray.DataArray
        distance of the each ice shelf point to the given line of interest
    """
    
    # add a common dimension 'grid' along which to stack
    stacked_isf_points = isf_points_da.stack(grid=['y', 'x'])
    stacked_line = line_points_da.stack(grid=['y', 'x'])
    
    # remove nans
    filtered_isf_points = stacked_isf_points[stacked_isf_points>0]
    filtered_line = stacked_line[stacked_line>0]
    
    # write out the y,x pairs behind the dimension 'grid'
    grid_isf_points = filtered_isf_points.indexes['grid'].to_frame().values.astype(float)
    grid_line = filtered_line.indexes['grid'].to_frame().values.astype(float)
    
    # create tree to line and compute distance
    tree_line = cKDTree(grid_line)
    dist_yx_to_line, _ = tree_line.query(grid_isf_points)
        
    # add the coordinates of the previous variables
    xr_dist_to_line = filtered_isf_points.copy(data=dist_yx_to_line)
    # put 1D array back into the format of the grid and put away the 'grid' dimension
    xr_dist_to_line = xr_dist_to_line.unstack('grid')
    
    return xr_dist_to_line

In [ ]:
dist_list = [ ]
for kisf in file_isf['Nisf']:

        if (file_isf['IF_mask']==kisf).sum() > 0:
            region_to_cut_out = close_region_around_isf_mask.sel(Nisf=kisf).squeeze()
            region_to_cut_out = region_to_cut_out.where(region_to_cut_out > 0, drop=True)
            IF_region = file_isf['IF_mask'].where(file_isf['IF_mask']==kisf, drop=True)

            dist_from_front = distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
            dist_list.append(dist_from_front)

dist_all = xr.concat(dist_list, dim='Nisf').reindex_like(file_isf)
dist_all.to_dataset(name='dist_from_front').to_netcdf(outputpath_profiles+'dist_to_ice_front_only_contshelf_8km.nc')